In [1]:
import pandas as pd
from datetime import timedelta


In [2]:
train_df = pd.read_csv('../data/raw/train.csv', parse_dates=['date'])
holiday_df = pd.read_csv('../data/raw/holidays_events.csv', parse_dates=['date'])
oil_df = pd.read_csv('../data/raw/oil.csv', parse_dates=['date'])
stores_df = pd.read_csv('../data/raw/stores.csv')
transactions_df = pd.read_csv('../data/raw/transactions.csv', parse_dates=['date'])
test_df = pd.read_csv('../data/raw/test.csv', parse_dates=['date'])
train_df.shape

(3000888, 6)

In [3]:
# print null rows of any dataframe
def print_null_rows(df, name):
    null_rows = df[df.isnull().any(axis=1)]
    if not null_rows.empty:
        print(f"shape:{df.shape}, Null rows in {name}:\n{null_rows}\n")
    else:
        print(f"shape:{df.shape}, No null rows in {name}.\n")
    
print_null_rows(train_df, 'train_df')
print_null_rows(holiday_df, 'holiday_df')
print_null_rows(oil_df, 'oil_df')
print_null_rows(stores_df, 'stores_df')
print_null_rows(transactions_df, 'transactions_df')
print_null_rows(test_df, 'test_df')

shape:(3000888, 6), No null rows in train_df.

shape:(350, 6), No null rows in holiday_df.

shape:(1218, 2), Null rows in oil_df:
           date  dcoilwtico
0    2013-01-01         NaN
14   2013-01-21         NaN
34   2013-02-18         NaN
63   2013-03-29         NaN
104  2013-05-27         NaN
132  2013-07-04         NaN
174  2013-09-02         NaN
237  2013-11-28         NaN
256  2013-12-25         NaN
261  2014-01-01         NaN
274  2014-01-20         NaN
294  2014-02-17         NaN
338  2014-04-18         NaN
364  2014-05-26         NaN
393  2014-07-04         NaN
434  2014-09-01         NaN
497  2014-11-27         NaN
517  2014-12-25         NaN
522  2015-01-01         NaN
534  2015-01-19         NaN
554  2015-02-16         NaN
588  2015-04-03         NaN
624  2015-05-25         NaN
653  2015-07-03         NaN
699  2015-09-07         NaN
757  2015-11-26         NaN
778  2015-12-25         NaN
783  2016-01-01         NaN
794  2016-01-18         NaN
814  2016-02-15         NaN
84

## 1. Darts Final Train without lagging and all date index must exist

In [4]:
# interpolate missing values in oil prices
all_dates = pd.date_range(start=oil_df['date'].min(), end=oil_df['date'].max())
oil_df = oil_df.set_index('date').reindex(all_dates).rename_axis('date').reset_index()
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].interpolate(method='polynomial', order=2)
# fill backward and forward fill for oil prices
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].bfill()

In [5]:
print_null_rows(train_df, 'train_df')
print_null_rows(holiday_df, 'holiday_df')
print_null_rows(oil_df, 'oil_df')
print_null_rows(stores_df, 'stores_df')
print_null_rows(transactions_df, 'transactions_df')
print_null_rows(test_df, 'test_df')

shape:(3000888, 6), No null rows in train_df.

shape:(350, 6), No null rows in holiday_df.

shape:(1704, 2), No null rows in oil_df.

shape:(54, 5), No null rows in stores_df.

shape:(83488, 3), No null rows in transactions_df.

shape:(28512, 5), No null rows in test_df.



In [6]:
# create faetures for train and test sets
def create_features(df):
    def days_since_payday(date):
        day = date.day
        if day <= 15:
            # Days since last month's end
            last_month_end = date.replace(day=1) - timedelta(days=1)
            return (date - last_month_end).days
        else:
            # Days since 15th of current month
            current_month_15th = date.replace(day=15)
            return (date - current_month_15th).days
        
    def days_until_payday(date):
        day = date.day
        if day < 15:
            # Days until 15th
            return 15 - day
        else:
            # Days until month end
            next_month = date.replace(day=28) + timedelta(days=4)
            month_end = next_month - timedelta(days=next_month.day)
            return (month_end - date).days
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['weekofyear'] = df['date'].dt.isocalendar().week
    df['day_of_year'] = df['date'].dt.dayofyear
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    df['is_quarter_start'] = df['date'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['date'].dt.is_quarter_end.astype(int)
    df['is_payday'] = ((df['day'] == 15) | df['date'].dt.is_month_end).astype(int)
    df['days_since_payday'] = df['date'].apply(days_since_payday)
    df['days_until_payday'] = df['date'].apply(days_until_payday)
    return df

def create_lag_features(df, target_col='sales', lags=[1, 7, 14, 30]):
    """Create lag features for time series"""
    df_sorted = df.sort_values(['store_nbr', 'family', 'date'])
    
    for lag in lags:
        df_sorted[f'{target_col}_lag_{lag}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].shift(lag)
    
    return df_sorted

def create_rolling_features(df, target_col='sales', windows=[7, 14, 30]):
    """Create rolling window statistics"""
    df_sorted = df.sort_values(['store_nbr', 'family', 'date'])
    
    for window in windows:
        # Rolling mean
        df_sorted[f'{target_col}_rolling_mean_{window}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        
        # Rolling std
        df_sorted[f'{target_col}_rolling_std_{window}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
        
        # Rolling max
        df_sorted[f'{target_col}_rolling_max_{window}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).max()
        )
        
        # Rolling min
        df_sorted[f'{target_col}_rolling_min_{window}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).min()
        )
    
    return df_sorted


In [7]:
train_df = create_features(train_df)
print_null_rows(train_df, 'train_df_temporal')

shape:(3000888, 20), No null rows in train_df_temporal.



In [8]:
train_df = create_rolling_features(train_df)
# print number of nulls in train_df_rolling
# fill nulls in rolling features with 0
print_null_rows(train_df, 'train_df_oil')
train_df.fillna(0, inplace=True)
print_null_rows(train_df, 'train_df_oil')

In [10]:
""" No lagging features because Darts Model handle them internally."""
# train_df = create_lag_features(train_df)
# train_df.isnull().sum()

' No lagging features because Darts Model handle them internally.'

In [12]:
national_holidays = holiday_df[holiday_df['locale'] == 'National']['date'].unique()
regional_holidays = holiday_df[holiday_df['locale'] == 'Regional']['date'].unique()
local_holidays = holiday_df[holiday_df['locale'] == 'Local']['date'].unique()
additional_holidays = holiday_df[holiday_df['type'] == 'Additional']['date'].unique()
working_days = holiday_df[holiday_df['type'] == 'Work Day']['date'].unique()
events = holiday_df[holiday_df['type'] == 'Event']['date'].unique()
bridge_days = holiday_df[holiday_df['type'] == 'Bridge']['date'].unique()
transsferred_days = holiday_df[holiday_df['transferred'] == True]['date'].unique()

In [13]:
# add holiday features to train
def add_holiday_features(df):
    df['is_national_holiday'] = df['date'].isin(national_holidays).astype(int)
    df['is_regional_holiday'] = df['date'].isin(regional_holidays).astype(int)
    df['is_local_holiday'] = df['date'].isin(local_holidays).astype(int)
    df['is_additional_holiday'] = df['date'].isin(additional_holidays).astype(int)
    df['is_working_day'] = df['date'].isin(working_days).astype(int)
    df['is_event'] = df['date'].isin(events).astype(int)
    df['is_bridge_day'] = df['date'].isin(bridge_days).astype(int)
    df['is_transferred_day'] = df['date'].isin(transsferred_days).astype(int)
    return df

# create features for train and test sets
train_df = add_holiday_features(train_df)
print_null_rows(train_df, 'train_df')

shape:(3000888, 40), No null rows in train_df.



In [14]:
train_df = train_df.merge(oil_df, on='date', how='left')
print_null_rows(train_df, 'train_df_final')

shape:(3000888, 41), No null rows in train_df_final.



In [15]:
train_df = train_df.merge(stores_df, on='store_nbr', how='left')
print_null_rows(train_df, 'train_df_final')

shape:(3000888, 45), No null rows in train_df_final.



In [16]:
train_df = train_df.merge(transactions_df, on=['date', 'store_nbr'], how='left')
print_null_rows(train_df, 'train_df_final')

shape:(3000888, 46), Null rows in train_df_final:
              id       date  store_nbr      family  sales  onpromotion  year  \
0              0 2013-01-01          1  AUTOMOTIVE    0.0            0  2013   
364       648648 2014-01-01          1  AUTOMOTIVE    0.0            0  2014   
728      1297296 2015-01-01          1  AUTOMOTIVE    0.0            0  2015   
915      1630530 2015-07-07          1  AUTOMOTIVE    0.0            0  2015   
1092     1945944 2016-01-01          1  AUTOMOTIVE    0.0            0  2016   
...          ...        ...        ...         ...    ...          ...   ...   
2999932  1298945 2015-01-01         54     SEAFOOD    0.0            0  2015   
3000296  1947593 2016-01-01         54     SEAFOOD    0.0            0  2016   
3000298  1951157 2016-01-03         54     SEAFOOD    2.0            0  2016   
3000299  1952939 2016-01-04         54     SEAFOOD    3.0            0  2016   
3000661  2598023 2017-01-01         54     SEAFOOD    0.0            0

In [17]:
# fill missing values in transactions with 0
train_df['transactions'] = train_df['transactions'].fillna(0)
print_null_rows(train_df, 'train_df_final')

shape:(3000888, 46), No null rows in train_df_final.



In [18]:
train_df.to_csv('../data/interim/Darts_final_train.csv', index=False)

## 2. Feature Engineering for Test Set (Avoiding Data Leakage)

In [19]:
# Create a copy of test_df for processing
test_processed = test_df.copy()

# Apply temporal features (safe - no data leakage)
test_processed = create_features(test_processed)
print_null_rows(test_processed, 'test_df_temporal')

shape:(28512, 19), No null rows in test_df_temporal.



In [20]:
# Add holiday features (safe - holidays are predetermined)
test_processed = add_holiday_features(test_processed)
print_null_rows(test_processed, 'test_df_holidays')

shape:(28512, 27), No null rows in test_df_holidays.



In [21]:
# Merge oil prices with test data
test_processed = test_processed.merge(oil_df, on='date', how='left')
print_null_rows(test_processed, 'test_df_oil')

shape:(28512, 28), No null rows in test_df_oil.



In [22]:
# Merge store information
test_processed = test_processed.merge(stores_df, on='store_nbr', how='left')
print_null_rows(test_processed, 'test_df_stores')

shape:(28512, 32), No null rows in test_df_stores.



In [23]:
# For transactions in test period, we need to be careful
# We can either:
# 1. Use 0 for all test transactions (conservative)
# 2. Use historical averages for each store
# 3. Predict transactions separately

# Option 2: Use historical averages
store_avg_transactions = transactions_df.groupby('store_nbr')['transactions'].mean().reset_index()
store_avg_transactions.columns = ['store_nbr', 'avg_transactions']

# Merge with test data
test_processed = test_processed.merge(store_avg_transactions, on='store_nbr', how='left')

# Use average transactions for test period
test_processed['transactions'] = test_processed['avg_transactions']
test_processed.drop('avg_transactions', axis=1, inplace=True)

print_null_rows(test_processed, 'test_df_transactions')

shape:(28512, 33), No null rows in test_df_transactions.



In [24]:
# For rolling features, we need to be very careful to avoid data leakage
# We can only compute rolling features using historical training data
# 
# Create rolling features using only training data up to each test date
def create_rolling_features_no_leakage(train_df, test_df, target_col='sales', windows=[7, 14, 30]):
    """Create rolling features for test set using only historical training data"""
    
    # Combine train and test for proper sorting, but only compute rolling on train
    # Add a temporary sales column to test (will be removed)
    test_with_temp_sales = test_df.copy()
    test_with_temp_sales[target_col] = 0  # Temporary placeholder
    
    # Combine datasets
    combined = pd.concat([train_df, test_with_temp_sales], ignore_index=True)
    combined = combined.sort_values(['store_nbr', 'family', 'date'])
    
    # Mark train vs test
    combined['is_train'] = combined['date'] <= train_df['date'].max()
    
    for window in windows:
        # Rolling mean - only computed on training data
        combined[f'{target_col}_rolling_mean_{window}'] = combined.groupby(['store_nbr', 'family']).apply(
            lambda group: group[target_col].where(group['is_train']).rolling(window=window, min_periods=1).mean().ffill()
        ).reset_index(level=[0, 1], drop=True)
        
        # Rolling std
        combined[f'{target_col}_rolling_std_{window}'] = combined.groupby(['store_nbr', 'family']).apply(
            lambda group: group[target_col].where(group['is_train']).rolling(window=window, min_periods=1).std().ffill()
        ).reset_index(level=[0, 1], drop=True)
        
        # Rolling max
        combined[f'{target_col}_rolling_max_{window}'] = combined.groupby(['store_nbr', 'family']).apply(
            lambda group: group[target_col].where(group['is_train']).rolling(window=window, min_periods=1).max().ffill()
        ).reset_index(level=[0, 1], drop=True)
        
        # Rolling min
        combined[f'{target_col}_rolling_min_{window}'] = combined.groupby(['store_nbr', 'family']).apply(
            lambda group: group[target_col].where(group['is_train']).rolling(window=window, min_periods=1).min().ffill()
        ).reset_index(level=[0, 1], drop=True)
    
    # Return only test portion
    test_result = combined[~combined['is_train']].copy()
    test_result = test_result.drop(['is_train', target_col], axis=1)  # Remove temporary columns
    
    return test_result

print("Creating rolling features for test data using only historical training data...")

Creating rolling features for test data using only historical training data...


In [25]:
# Apply rolling features without data leakage
test_processed = create_rolling_features_no_leakage(train_df, test_processed)

# Fill any remaining NaN values with 0
test_processed = test_processed.fillna(0)

print_null_rows(test_processed, 'test_df_final')
print(f"Test processed shape: {test_processed.shape}")
print(f"Test processed columns: {list(test_processed.columns)}")

C:\Users\Fady Adel\AppData\Local\Temp\ipykernel_3192\2004204811.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined[f'{target_col}_rolling_mean_{window}'] = combined.groupby(['store_nbr', 'family']).apply(
C:\Users\Fady Adel\AppData\Local\Temp\ipykernel_3192\2004204811.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined[f'{target_col}_rolling_std_{window}'] = combined.groupby(['store_nbr

shape:(28512, 45), No null rows in test_df_final.

Test processed shape: (28512, 45)
Test processed columns: ['id', 'date', 'store_nbr', 'family', 'onpromotion', 'year', 'month', 'day', 'dayofweek', 'weekofyear', 'day_of_year', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_payday', 'days_since_payday', 'days_until_payday', 'sales_rolling_mean_7', 'sales_rolling_std_7', 'sales_rolling_max_7', 'sales_rolling_min_7', 'sales_rolling_mean_14', 'sales_rolling_std_14', 'sales_rolling_max_14', 'sales_rolling_min_14', 'sales_rolling_mean_30', 'sales_rolling_std_30', 'sales_rolling_max_30', 'sales_rolling_min_30', 'is_national_holiday', 'is_regional_holiday', 'is_local_holiday', 'is_additional_holiday', 'is_working_day', 'is_event', 'is_bridge_day', 'is_transferred_day', 'dcoilwtico', 'city', 'state', 'type', 'cluster', 'transactions']


In [26]:
# Save the processed test data
test_processed.to_csv('../data/interim/Darts_final_test.csv', index=False)
print("Test data with features saved to '../data/interim/Darts_final_test.csv'")

# Verify column consistency between train and test
train_cols = set(train_df.columns) - {'sales'}  # Remove target column
test_cols = set(test_processed.columns)

missing_in_test = train_cols - test_cols
extra_in_test = test_cols - train_cols

print(f"\nColumn consistency check:")
print(f"Missing in test: {missing_in_test}")
print(f"Extra in test: {extra_in_test}")

if len(missing_in_test) == 0 and len(extra_in_test) == 0:
    print("✅ Perfect column alignment between train and test datasets!")
else:
    print("⚠️  Column mismatch detected. Please review.")

Test data with features saved to '../data/interim/Darts_final_test.csv'

Column consistency check:
Missing in test: set()
Extra in test: set()
✅ Perfect column alignment between train and test datasets!
